In [2]:
fege

NameError: name 'fege' is not defined

In [12]:
import sys
sys.path.append('../_amigocloud')

from sqlalchemy import create_engine, text
from amigocloud import AmigoCloud
from docx import Document
from docx2pdf import convert
import os
import json
import pandas as pd

# CONEXIÓN A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

gauth = GoogleAuth()

# Si existe el archivo, intenta cargarlo
if os.path.exists("mycreds.txt"):
    gauth.LoadCredentialsFile("mycreds.txt")

# Si no hay credenciales o son inválidas, forzar nueva autenticación
if gauth.credentials is None or gauth.access_token_expired:
    try:
        gauth.LocalWebserverAuth()
    except:
        print("Error de autenticación. Eliminá 'mycreds.txt' y volvé a intentarlo.")
else:
    gauth.Authorize()

gauth.SaveCredentialsFile("mycreds.txt")
drive = GoogleDrive(gauth)

In [4]:
sys.path.append('..')

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA
RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [5]:
def get_ruta_informes():
    try:
        with open('../../ruta_informes.txt', 'r') as file:
            ruta_inf = file.readline().strip()
            return ruta_inf
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [6]:
ruta_informes = get_ruta_informes()
directorio_origen = ruta_informes + r'\_informes'
directorio_destino = ruta_informes + r'\_pdf'

In [7]:
# Recorre y convierte a pdf archivos .docx que no empiezan con "_"
def convertir_docx_to_pdf():
    for archivo in os.listdir(directorio_origen):
        if archivo.lower().endswith(".docx") and not archivo.startswith("_"):
            ruta_docx = os.path.join(directorio_origen, archivo)
            ruta_pdf = os.path.join(directorio_destino, archivo.replace(".docx", ".pdf"))
            
            try:
                convert(ruta_docx, ruta_pdf)
                print(ruta_docx)
                if os.path.exists(ruta_pdf):
                    print(f"✅ Convertido: {archivo}")
                    os.remove(ruta_docx)
                    print(f"🗑️ Eliminado original: {archivo}")
                else:
                    print(f"❌ Error: PDF no generado para {archivo}")
            except Exception as e:
                print(f"⚠️ Error al convertir {archivo}: {e}")

def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def cargar_link_to_db(url, cite):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.resumen_control_bio SET link = :url WHERE cite = :cite
            """)
            conn.execute(query, {"url": url, "cite": cite})
            print(f"✔️ Se registro link para CITE: {cite}" )
    except Exception as e:
        print(f"❌ Error al actualizar CITE: {cite}, {e}")

def get_registros_sin_agrocittca():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_control_bio.resumen_control_bio
            WHERE agrocittca = false
        """
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"❌ Error al consultar unidades sin labor: {e}")
        return pd.DataFrame()
    return None

In [6]:
convertir_docx_to_pdf()

In [8]:
def cargar_pdf_to_drive(id_drive, title_file, path_file):
    try:
        # 1. Buscar si ya existe un archivo con el mismo nombre en la misma carpeta
        query = f"'{id_drive}' in parents and title = '{title_file}' and trashed = false"
        file_list = drive.ListFile({'q': query}).GetList()

        if file_list:
            # Si ya existe, usamos el primero encontrado
            file = file_list[0]
            print(f"[INFO] El archivo '{title_file}' ya existe. Se reemplazará.")
        else:
            # Si no existe, creamos uno nuevo
            file = drive.CreateFile({
                "mimeType": "application/pdf",
                "parents": [{"kind": "drive#fileLink", "id": id_drive}]
            })
            file['title'] = title_file
            print(f"[INFO] El archivo '{title_file}' no existe. Se creará uno nuevo.")

        # 2. Reemplazar el contenido
        file.SetContentFile(path_file)
        file.Upload()

        # 3. Dar permisos públicos si es nuevo
        if not file_list:
            file.InsertPermission({
                'type': 'anyone',
                'value': 'anyone',
                'role': 'reader'
            })

        print(f"[OK] Archivo '{title_file}' subido exitosamente.")
        return file['alternateLink']

    except Exception as e:
        print(f"[ERROR] No se pudo subir el archivo '{title_file}': {e}")
        return False

In [9]:
contenido = os.listdir(directorio_destino)
contenido

[]

In [10]:
for file in contenido:
    partes = file.split('_')
    
    cite = partes[-1].split('.')[0]
    sigla = partes[1]
    title_file = file
    id_drive = '1sEfhWGATb7tkKgr6gcknH3ZA_ed_hGTq'
    path_file = os.path.join(directorio_destino, file)
    print(cite)
    url_drive = cargar_pdf_to_drive(id_drive, title_file, path_file)
    if url_drive == False:
        continue
    cargar_link_to_db(url_drive, cite)
    if os.path.exists(path_file):
        #os.remove(path_file)
        print("✅ Archivo eliminado.")
    else:
        print("⚠️ El archivo no existe.")

In [13]:
get_registros_sin_agrocittca()

,id,fecha,institucion,cod_canero,nombre_canero,hectareas,cite,cod_propiedad,propiedad,producto,link,agrocittca
0,6,2025-07-14,62,13171,SUAREZ JUSTINIANO DAVID,3.124797,CB-2025-12,1588,58--SUAREZ DAVID,TRICHOGRAMMA,https://drive.google.com/file/d/1aSqCHtlRyW3c6...,False
1,4,2025-07-09,124,1530,C.I.T.T.C.A.,39.205795,CB-2025-10,299,CITTCA,CRISOPA,https://drive.google.com/file/d/1tgRNiJUSlgK2B...,False
2,16,2025-06-26,124,1530,C.I.T.T.C.A.,185.835121,CB-2025-4,299,CITTCA,CRISOPA,https://drive.google.com/file/d/1D_st-sER0031n...,False
3,5,2025-06-28,124,1530,C.I.T.T.C.A.,11.004916,CB-2025-11,2257,TODOS SANTOS--CITTCA,CRISOPA,https://drive.google.com/file/d/1uRJ1uU9gKehR0...,False
4,10,2025-07-21,50,1672,CAMPBELL BALCAZAR ALAN,152.178669,CB-2025-16,1454,MIRAFLORES Y FAMOSA--CAMPBELL FERNANDO,TRICHOGRAMMA,https://drive.google.com/file/d/1JZCAatRoa90AR...,False
5,7,2025-07-14,69,18189,BALCAZAR SOLETO RAMON BERTHY,1.893873,CB-2025-13,1818,EL PUESTO--BALCAZAR RAMON,TRICHOGRAMMA,https://drive.google.com/file/d/1UGcB0S72QSbhr...,False
6,8,2025-07-14,69,18189,BALCAZAR SOLETO RAMON BERTHY,4.112150,CB-2025-14,1694,EL TRILLO--BALCAZAR RAMON,TRICHOGRAMMA,https://drive.google.com/file/d/16RPn5z6SUafgX...,False
7,19,2025-07-14,69,18358,ROJAS AMELUNGE CARLOS EDUARDO,58.338704,CB-2025-7,19,SANTA BARBARA,TRICHOGRAMMA,https://drive.google.com/file/d/1hwgeq0DKLpF8U...,False
8,21,2025-07-14,69,18358,ROJAS AMELUNGE CARLOS EDUARDO,7.675120,CB-2025-9,218,DON QUIJOTE,TRICHOGRAMMA,https://drive.google.com/file/d/10lilhfnmv2XBK...,False
9,20,2025-07-14,69,388,AGROPECUARIA MARIANA S.R.L.,4.597557,CB-2025-8,218,DON QUIJOTE,TRICHOGRAMMA,https://drive.google.com/file/d/165ciIYLwG3oS-...,False
